In [1]:
import sys
# sys.path.append('pytorchvideo')
from pytorchvideo.data import LabeledVideoDataset

In [2]:
from glob import glob
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

In [3]:
%pwd

'/workspace/persistent/Genuine_Posed_EmotionRecognition/video_classification'

# Create data

In [4]:
# # Create the train_df
# fake_angry = glob('train_root/fake_angry/*')
# fake_contempt = glob('train_root/fake_contempt/*')
# fake_disgust = glob('train_root/fake_disgust/*')
# fake_happy = glob('train_root/fake_happy/*')
# fake_sad = glob('train_root/fake_sad/*')
# fake_surprise = glob('train_root/fake_surprise/*')
# real_angry = glob('train_root/real_angry/*')
# real_contempt = glob('train_root/real_contempt/*')
# real_disgust = glob('train_root/real_disgust/*')
# real_happy = glob('train_root/real_happy/*')
# real_sad = glob('train_root/real_sad/*')
# real_surprise = glob('train_root/real_surprise/*')


# label = [0]*len(fake_angry) + [1]*len(fake_contempt) + [2]*len(fake_disgust) + [3]*len(fake_happy) + [4]*len(fake_sad) + [5]*len(fake_surprise) + [6]*len(real_angry) + [7]*len(real_contempt) + [8]*len(real_disgust) + [9]*len(real_happy) + [10]*len(real_sad) + [11]*len(real_surprise)

# # Convert this into dataframe
# train_df = pd.DataFrame(zip(fake_angry + fake_contempt + fake_disgust + fake_happy + fake_sad + fake_surprise + real_angry + real_contempt + real_disgust + real_happy + real_sad + real_surprise, label), columns=['file', 'label'])
# train_df.head()

# Augmentation Process

In [5]:
# Augmentation process
from pytorchvideo.data import LabeledVideoDataset, make_clip_sampler, labeled_video_dataset


from pytorchvideo.transforms import (
    ApplyTransformToKey,
    Normalize,
    RandomShortSideScale,

    UniformTemporalSubsample,
    Permute
)

from torchvision.transforms import (
    Compose,
    Lambda,
    RandomCrop,
    RandomHorizontalFlip,
    Resize
)

from torchvision.transforms._transforms_video import (
    CenterCropVideo,
    NormalizeVideo,
)

/opt/conda/lib/python3.9/site-packages/torchvision/transforms/functional_tensor.py:5: UserWarning: The torchvision.transforms.functional_tensor module is deprecated in 0.15 and will be **removed in 0.17**. Please don't rely on it. You probably just need to use APIs in torchvision.transforms.functional or in torchvision.transforms.v2.functional.
  warnings.warn(
/opt/conda/lib/python3.9/site-packages/torchvision/transforms/_functional_video.py:6: UserWarning: The 'torchvision.transforms._functional_video' module is deprecated since 0.12 and will be removed in the future. Please use the 'torchvision.transforms.functional' module instead.
  warnings.warn(
/opt/conda/lib/python3.9/site-packages/torchvision/transforms/_transforms_video.py:22: UserWarning: The 'torchvision.transforms._transforms_video' module is deprecated since 0.12 and will be removed in the future. Please use the 'torchvision.transforms' module instead.
  warnings.warn(


In [6]:
video_transform = Compose([
    ApplyTransformToKey(key='video',
        transform = Compose([
        UniformTemporalSubsample(20),
        Lambda(lambda x:x/255),
        NormalizeVideo(mean=[0.45, 0.45, 0.45], std=[0.225, 0.225, 0.225]),
        RandomShortSideScale(min_size=256, max_size=256),
        CenterCropVideo(224),
        RandomHorizontalFlip(p=0.5)
    ]),
    ),
])

# Train Data Loader

In [7]:
from torch.utils.data import DataLoader

# Labeled video dataset
train_dataset = labeled_video_dataset('../data_temporal/train_root',
                                      clip_sampler=make_clip_sampler('random', 2),
                                        transform = video_transform, decode_audio=False)


train_loader = DataLoader(train_dataset, batch_size=5, num_workers=0, pin_memory=True)

In [8]:
train_batch = next(iter(train_loader))

In [9]:
train_batch.keys()

dict_keys(['video', 'video_name', 'video_index', 'clip_index', 'aug_index', 'label'])

In [10]:
train_batch['video'].shape, train_batch['label'].shape


(torch.Size([5, 3, 20, 224, 224]), torch.Size([5]))

In [11]:
# Validation Data Loader

In [12]:
val_dataset = labeled_video_dataset('../data_temporal/val_root',
                                      clip_sampler=make_clip_sampler('random', 2),
                                        transform = video_transform, decode_audio=False)


val_loader = DataLoader(train_dataset, batch_size=5, num_workers=0, pin_memory=True)

In [13]:
val_batch = next(iter(val_loader))

In [14]:
val_batch.keys()

dict_keys(['video', 'video_name', 'video_index', 'clip_index', 'aug_index', 'label'])

In [15]:
val_batch['video'].shape, val_batch['label'].shape

(torch.Size([5, 3, 20, 224, 224]), torch.Size([5]))

# NEXT STEP

In [16]:
# TODO....